## OWSLib

[OGC OWSlib](http://geopython.github.io/OWSLib) is interface from the Python language for OGC OWS. The library enables you to connect to the services as client. Currently WMS, WFS, WCS, CSW, WPS, SOS, WMC and others (the list is growing). 

Documentation: http://geopython.github.io/OWSLib/


### OGC CSW

First think you need, when you want to use some web service, is to know, where it is located. For this purpose, catalogues are build - special services with list of avaliable data, services and application out there - actually list of their metadata. OGC CSW is used very much in european INSPIRE directive and the main entry point for Czech national INSPIRE geoportal is located at http://geoportal.gov.cz/php/micka/csw/index.php (web interface can be found at http://inspire.gov.cz).

Let's read some metadata

In [ ]:
from owslib.csw import CatalogueServiceWeb
thecsw = CatalogueServiceWeb('http://geoportal.gov.cz/php/micka/csw/index.php')
print(thecsw.service)

Download the records

In [ ]:
thecsw.getrecords2()
print(thecsw.results)

Check out some attributes of found records:

In [ ]:
for rec in thecsw.records:
    print(thecsw.records[rec].title)

### Filters

With CSW, you can use another standard of OGC: Filter Encoding Sepcification (FES). Filters can use attributes or geometry. In following example, we will define two filters: records with keyword 'WMS' and with data covering city Prague.

In [ ]:
from owslib.fes import PropertyIsLike, BBox, And, PropertyIsEqualTo

wms_query = PropertyIsEqualTo('csw:AnyText', 'WMS')
praha_query = BBox([14.22,49.94,14.71,50.18])
praha_and_wms = And([praha_query, wms_query])

thecsw.getrecords2([praha_and_wms], esn='full')
print(thecsw.results)

for recid in thecsw.records:
    record = thecsw.records[recid]
    print(record.title, record.bbox.minx, record.bbox.miny, record.bbox.maxx, record.bbox.maxy)

Find out attributes of metadata record:

In [ ]:
zm_query = PropertyIsEqualTo('csw:AnyText', 'ZM10')
thecsw.getrecords2([zm_query], esn='full')
zm10 = thecsw.records['CZ-CUZK-WMS-ZM10-P']

print("Service: %s" % zm10.type)

print("Title: %s" % zm10.title)

print("Abstract: %s" % zm10.abstract)

zm10_url = zm10.references[0]['url']

print("Service end point: %s" % zm10_url)

### OGC WMS

OGC Web Map Service is there for downloading rendered images of the map.

In [ ]:
from owslib.wms import WebMapService

useragent="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"

zm10_wms = WebMapService(zm10_url, version="1.3.0", headers={"User-Agent":useragent})

print("WMS title: %s" % zm10_wms.identification.title)

print("WMS abstract: %s" % zm10_wms.identification.abstract)

print("Provider name: %s" % zm10_wms.provider.name)

print("Provider address: %s" % zm10_wms.provider.contact.address)

Available WMS layers:

In [ ]:
print(zm10_wms.contents)

print(zm10_wms.contents['GR_ZM10'].boundingBox)

print(zm10_wms.contents['GR_ZM10'].boundingBoxWGS84)

Download and save the data (image)

In [ ]:
img = zm10_wms.getmap(layers=['GR_ZM10'],
    size=[800, 600],
    srs="EPSG:5514",
    bbox=[-950003.175021186, -1250003.1750036045, -399990.474995786, -899996.8249909044],
    format="image/png",
    headers={"User-Agent":useragent})

out = open('../../outputs/zm10.png', 'w')
out.write(img.read())
out.close()

In [ ]:
from IPython.display import Image
Image(filename='zm10.png')

### OGC WFS

OGC Web Feature Service deals for downloading of vector data. Usually OGC GML format is used for data exchange.

First step: let's find some WFS service in our CSW server:

In [ ]:
wfs_query = PropertyIsLike('csw:AnyText', 'WFS')
aopk_query = PropertyIsLike('csw:AnyText', 'AOPK')
service_query = PropertyIsLike('apiso:type', 'service')
aopk_and_wfs = And([aopk_query, wfs_query, service_query])
thecsw.getrecords2([aopk_and_wfs], esn='full')

print(thecsw.results)

for recid in thecsw.records:
    record = thecsw.records[recid]
    print(recid, record.title)

Let's have a look at WFS data in Nature Conservation Agency of the Czech Republic

In [ ]:
natura = thecsw.records['53e37222-89a0-472b-9781-5bfc0a02080a']
print("Abstract:\n %s\n" % natura.abstract)

print("Identifier: %s" % natura.identifiers[1]['identifier'])

Let's read some data

In [ ]:
from owslib import wfs as webfeatureservice
url = natura.identifiers[1]['identifier']
# https does not work, use
aopk = webfeatureservice.WebFeatureService('http://gis.nature.cz/arcgis/services/UzemniOchrana/Natura2000/MapServer/WFSServer')

Zjistíme vlastnosti služby (Capabilities):

In [ ]:
capabilities = aopk.getcapabilities()
print("URL: %s" % capabilities.geturl())
print("Name: %s" % aopk.provider.name)
print("Title: %s" % aopk.identification.title)
print("Keywords: %s" % aopk.identification.keywords[0])
print("Fees: %s" % aopk.identification.fees)
print("Abstract: %s" % aopk.identification.abstract)

Metadata attributes of the service

In [ ]:
for i in aopk.contents:
    print(i)

Get the data now

In [ ]:
identifier = 'UzemniOchrana_Natura2000:Evropsky_významná_lokalita__EVL_'
features = aopk.getfeature([identifier])
print(features)

Print the XML nicely

In [ ]:
print(features.read())

### State Administration of Land Surveying and Cadastre WFS service

another example: NUTS3 regions from the official WFS server of State Administration of Land Surveying and Cadastre

In [ ]:
cuzk = webfeatureservice.WebFeatureService('http://geoportal.cuzk.cz/wfs_au/wfservice.aspx', version="2.0.0", headers={"User-agent":useragent})
for c in cuzk.contents:
    print(c)

nuts3 = cuzk.getfeature(typename=['gmgml:KRAJ'])
nuts3.read() # this seems to be broken right now :-/